# Second virial coefficient calculation of TRAPPE ethane using Mayer-Sampling


Here, we reproduce the virial coefficient of TRAPPE ethane

Table 6 of https://doi.org/10.1021/jp972543+ reports −73 +/- 1 ml/mol at 423.2K

In [3]:
import unittest
import feasst as fst

class TestMayerSamplingTRAPPEethane(unittest.TestCase):
    def test(self):
        mc = fst.MonteCarlo()
        config = fst.MakeConfiguration(fst.args({"cubic_box_length": str(fst.NEAR_INFINITY)}))
        config.add_particle_type("data.ethane_with_ref")
        config.add_particle_type("data.ethane_with_ref", "2")
        config.add_particle_of_type(0)
        config.add_particle_of_type(1)
        for site in range(config.num_site_types()):
            config.set_model_param("cutoff", site, 1e4)
        mc.add(config)
        mc.add(fst.MakePotential(fst.MakeLennardJones()))

        # reference
        ref = fst.MakePotential(fst.MakeHardSphere())
        ref_params = fst.ModelParams(mc.configuration().model_params())
        for itype in range(mc.configuration().num_site_types()):
            ref_params.set("sigma", itype, 0)
        for itype in [0, 2]:
            for jtype in [0, 2]:
                ref_params.set("sigma", itype, jtype, 4)
        ref.set(ref_params)
        mc.add_to_reference(ref)

        mc.set(fst.MakeThermoParams(fst.args({"beta": str(1./423.2)}))) # 1/K
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
        mc.add(fst.MakeTrialTranslate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "0.1", "particle_type": "1"})))
        mc.add(fst.MakeTrialRotate(fst.args({"new_only": "true", "reference_index": "0",
            "tunable_param": "40"})))
        steps_per = "1e5"
        mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "ethane"})))
        mc.add(fst.MakeTune(fst.args({"steps_per": steps_per})))
        mc.attempt(int(1e5))
        mc.perform(fst.MakeRemoveModify(fst.args({"name": "Tune"})))
        mc.attempt(int(1e6))
        mayer = fst.MakeMayerSampling()
        mc.set(mayer)
        mc.attempt(int(1e7))
        b2hs = 2./3.*fst.PI*mc.configuration().model_params().sigma().value(0)**3  # A^3
        b2hs *= 1e-30*1e3*1e3*mc.configuration().physical_constants().avogadro_constant() # L/mol
        print('mayer', mayer.mayer().str())
        print('mayer_ref', mayer.mayer_ref().str())
        print('b22(L/mol)', b2hs*mayer.second_virial_ratio())
        print('b22_block_stdev(L/mol)', b2hs*mayer.second_virial_ratio_block_stdev())
        self.assertAlmostEqual(-63, b2hs*mayer.second_virial_ratio(),
                               delta=3*b2hs*mayer.second_virial_ratio_block_stdev())

If the test passes, the energy is within the tolerance of the SRSW value and the two ensemble average methods agreed.

In [4]:
%time  # Note: any line starting with % is only to be used with ipynb
unittest.main(argv=[''], verbosity=2, exit=False)

test (__main__.TestMayerSamplingTRAPPEethane) ... 

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 12.4 µs
mayer average,stdev,block_stdev,n,moment0,moment1,
0.31049120000000002,0.95057630155716089,0.022292153840626105,10000000,3104912,10000000,
mayer_ref average,stdev,block_stdev,n,moment0,moment1,
-0.31432066586921176,0.73509937945137171,0.011593521302031978,10000000,-3143206.6586921174,6391685.2462514667,
b22(L/mol) -65.70216479676557
b22_block_stdev(L/mol) 5.303260340558539


ok

----------------------------------------------------------------------
Ran 1 test in 19.510s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!